In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
from collections import defaultdict
from sklearn.metrics import confusion_matrix
import warnings

from scipy.stats import pearsonr

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.feature_selection import RFE

from skimage.morphology import skeletonize, square, dilation 
from PVBM.GeometricalAnalysis import GeometricalVBMs
from PIL import Image,ImageFilter
from glob import glob

from itertools import combinations

In [2]:
img_size = (300, 300)
batch_size = 50

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(brightness=0.5, contrast=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.3998, 0.1676, 0.0636], [0.2762, 0.1356, 0.0666])
    ]),
    'val': transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize([0.3998, 0.1676, 0.0636], [0.2762, 0.1356, 0.0666])
    ]),
    'test': transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize([0.3998, 0.1676, 0.0636], [0.2762, 0.1356, 0.0666])
    ])
}

data_transforms_vessel = {
    'train': transforms.Compose([
        transforms.Resize(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor()        
    ]),
    'val': transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor()        
    ]),
    'test': transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor()        
    ])
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = model_ft = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=False)
num_ftrs = model_ft.classifier.in_features
model_ft.classifier  = nn.Sequential(nn.Linear(num_ftrs, 500),nn.Linear(500,  2))

Using cache found in C:\Users\micha/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\micha\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\micha\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [6]:
for typ in ['_vessel', '', '_prep']:
    data_dir = './data/data_m'+typ
    if typ == '_vessel': trans = data_transforms_vessel
    else: trans = data_transforms
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), trans[x]) for x in ['train', 'val', 'test']}
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=False, num_workers=4) for x in ['train', 'val', 'test']}

    model_ft.load_state_dict(torch.load('models/densenet121'+typ+'_e20_s300_b14.pt'))
    model_ft.to(device)
    model_ft.eval()

    x, y = {}, {}
    for set in ['train', 'test']:
        for i, data in enumerate(dataloaders[set], 0):
            samples, labels = data
            with torch.no_grad():
                samples = samples.to(device)
                output = model_ft.features(samples)
                output = nn.AdaptiveAvgPool3d(output_size=(1024,1,1))(output).squeeze()
                if i == 0: x[set], y[set] = torch.Tensor.numpy(output.cpu()), torch.Tensor.numpy(labels)
                else:
                    x[set] = np.concatenate((x[set], torch.Tensor.numpy(output.cpu())))
                    y[set] = np.concatenate((y[set], torch.Tensor.numpy(labels)))

    print('FOREST'+typ)                    
    forest = RandomForestClassifier(n_estimators=750, max_depth=6)
    forest.fit(x['train'], y['train'])

    prediction = forest.predict(x['train'])
    print('Training accuracy:', accuracy_score(y['train'], prediction))
    print('Training recall:', recall_score(y['train'], prediction))
    print('Training precistion:', precision_score(y['train'], prediction))
    print('Training F1: ', f1_score(y['train'], prediction), '\n')
    prediction = forest.predict(x['test'])
    print('Testing accuracy:', accuracy_score(y['test'], prediction))
    print('Testing recall:', recall_score(y['test'], prediction))
    print('Testing precistion:', precision_score(y['test'], prediction))
    print('Testing F1: ', f1_score(y['test'], prediction), '\n')
    
    sc = StandardScaler()
    x['train']= sc.fit_transform(x['train'])
    x['test'] = sc.transform(x['test'])

    print('REGRESTION'+typ)
    
    reg = LogisticRegression(solver='liblinear', max_iter=1000)
    reg.fit(x['train'], y['train'])

    prediction = reg.predict(x['train'])
    print('Training accuracy:', accuracy_score(y['train'], prediction))
    print('Training recall:', recall_score(y['train'], prediction))
    print('Training precistion:', precision_score(y['train'], prediction))
    print('Training F1: ', f1_score(y['train'], prediction), '\n')
    prediction = reg.predict(x['test'])
    print('Testing accuracy:', accuracy_score(y['test'], prediction))
    print('Testing recall:', recall_score(y['test'], prediction))
    print('Testing precistion:', precision_score(y['test'], prediction))
    print('Testing F1: ', f1_score(y['test'], prediction), '\n')
    
    print('SELECTED')
    
    sfs1 = RFE(reg, n_features_to_select=0.0833)
    sfs1 = sfs1.fit(x['train'], y['train'])
    
    X_train_sele = sfs1.transform(x['train'])
    X_test_sele = sfs1.transform(x['test'])
    
    reg.fit(X_train_sele, y['train'])
    
    prediction = reg.predict(X_train_sele)
    print('Training accuracy:', accuracy_score(y['train'], prediction))
    print('Training recall:', recall_score(y['train'], prediction))
    print('Training precistion:', precision_score(y['train'], prediction))
    print('Training F1: ', f1_score(y['train'], prediction), '\n')
    prediction = reg.predict(X_test_sele)
    print('Testing accuracy:', accuracy_score(y['test'], prediction))
    print('Testing recall:', recall_score(y['test'], prediction))
    print('Testing precistion:', precision_score(y['test'], prediction))
    print('Testing F1: ', f1_score(y['test'], prediction), '\n')

    print('KNN'+typ)
    
    knn = KNeighborsClassifier()
    knn.fit(x['train'], y['train'])\

    prediction = knn.predict(x['train'])
    print('Training accuracy:', accuracy_score(y['train'], prediction))
    print('Training recall:', recall_score(y['train'], prediction))
    print('Training precistion:', precision_score(y['train'], prediction))
    print('Training F1: ', f1_score(y['train'], prediction), '\n')
    prediction = knn.predict(x['test'])
    print('Testing accuracy:', accuracy_score(y['test'], prediction))
    print('Testing recall:', recall_score(y['test'], prediction))
    print('Testing precistion:', precision_score(y['test'], prediction))
    print('Testing F1: ', f1_score(y['test'], prediction), '\n')

    print('SELECTED')

    knn.fit(X_train_sele, y['train'])

    prediction = knn.predict(X_train_sele)
    print('Training accuracy:', accuracy_score(y['train'], prediction))
    print('Training recall:', recall_score(y['train'], prediction))
    print('Training precistion:', precision_score(y['train'], prediction))
    print('Training F1: ', f1_score(y['train'], prediction), '\n')
    prediction = knn.predict(X_test_sele)
    print('Testing accuracy:', accuracy_score(y['test'], prediction))
    print('Testing recall:', recall_score(y['test'], prediction))
    print('Testing precistion:', precision_score(y['test'], prediction))
    print('Testing F1: ', f1_score(y['test'], prediction), '\n')

FOREST_vessel
Training accuracy: 1.0
Training recall: 1.0
Training precistion: 1.0
Training F1:  1.0 

Testing accuracy: 0.8142857142857143
Testing recall: 0.8285714285714286
Testing precistion: 0.8055555555555556
Testing F1:  0.8169014084507044 

REGRESTION_vessel
Training accuracy: 1.0
Training recall: 1.0
Training precistion: 1.0
Training F1:  1.0 

Testing accuracy: 0.8428571428571429
Testing recall: 0.8285714285714286
Testing precistion: 0.8529411764705882
Testing F1:  0.8405797101449276 

SELECTED
Training accuracy: 0.987152034261242
Training recall: 0.9870689655172413
Training precistion: 0.9870689655172413
Training F1:  0.9870689655172413 

Testing accuracy: 0.8428571428571429
Testing recall: 0.8285714285714286
Testing precistion: 0.8529411764705882
Testing F1:  0.8405797101449276 

KNN_vessel
Training accuracy: 0.9486081370449678
Training recall: 0.9568965517241379
Training precistion: 0.940677966101695
Training F1:  0.9487179487179486 

Testing accuracy: 0.8285714285714286
Te